In [ ]:
import torch
import torch.nn as nn
from utils import *
import numpy as np
from dataset import *
from tqdm import tqdm
import torch.nn.functional as F
import os
from tools import *

In [ ]:
class Unet3D(nn.Module):
    def __init__(self, in_channels: int, 
                 n_feat: int = 256, 
                 n_cfeat: int = 10,
                 height: int = 28):
        super(Unet3D, self).__init__()

        self.in_channels = in_channels
        self.n_feat = n_feat
        self.n_cfeat = n_cfeat
        self.height = height

        self.init_conv = DoubleConv3D(in_channels, n_feat, is_res=True)

        self.down1 = UnetDown3D(n_feat, n_feat)
        self.down2 = UnetDown3D(n_feat, 2 * n_feat)

        self.to_vec = nn.Sequential(nn.AvgPool3d((self.height//4)), nn.GELU()) # convect down2 to vector

        self.timeembed1 = EmbedFC(1, 2*n_feat)
        self.timeembed2 = EmbedFC(1, 1*n_feat)

        self.contextembed1 = EmbedFC(n_cfeat, 2*n_feat)
        self.contextembed2 = EmbedFC(n_cfeat, 1*n_feat)

        self.up0 = nn.Sequential(
            nn.ConvTranspose3d(2 * n_feat, 2 * n_feat, self.height//4, self.height//4),
            nn.GroupNorm(8, 2 * n_feat),
            nn.ReLU()
        )

        self.up1 = UnetUp3D(4 * n_feat, n_feat)
        self.up2 = UnetUp3D(2 * n_feat, n_feat)

        self.out = nn.Sequential(
            nn.Conv3d(2 * n_feat, n_feat, 3, 1, 1),
            nn.GroupNorm(8, n_feat),
            nn.ReLU(),
            nn.Conv3d(n_feat, self.in_channels, 3, 1, 1)
        )

    def forward(self, x, t, c=None):


        x = self.init_conv(x)
        
        down1 = self.down1(x)
        
        down2 = self.down2(down1)
        
        hiddenvec = self.to_vec(down2)

        if c is None:
            c = torch.zeros(x.shape[0], self.n_cfeat).to(x)
        
        cemb1 = self.contextembed1(c).view(-1, self.n_feat * 2, 1, 1, 1)     # (batch, 2*n_feat, 1,1)
        temb1 = self.timeembed1(t).view(-1, self.n_feat * 2, 1, 1, 1)
        cemb2 = self.contextembed2(c).view(-1, self.n_feat, 1, 1, 1)
        temb2 = self.timeembed2(t).view(-1, self.n_feat, 1, 1, 1)

        up1 = self.up0(hiddenvec)
        
        up2 = self.up1(cemb1*up1 + temb1, down2)  # add and multiply embeddings
        
        up3 = self.up2(cemb2*up2 + temb2, down1)

        out = self.out(torch.cat((up3, x), 1))

        return out

In [ ]:
#hyperparameters

# For diffusion
timesteps = 1000
beta1 = 1e-4
beta2 = 0.02

# For network
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_feat = 64 # 64 hidden dimension feature
n_cfeat = 5 # context vector is of size 5
height = 44
save_dir = './weights/'

# training hyperparameters
n_epoch = 100
lrate=5e-3

In [ ]:
# DDPM noise schedule
b_t = (beta2 - beta1) * torch.linspace(0, 1, timesteps + 1, device=device) + beta1
a_t = 1 - b_t
ab_t = torch.cumsum(a_t.log(), dim=0).exp()    
ab_t[0] = 1

In [ ]:
# initialize model

nn_model = Unet3D(in_channels=1, n_feat=n_feat, n_cfeat=n_cfeat, height=height).to(device)

# Training

In [ ]:
# initialize dataset
data = torch.tensor(np.load('datasets/Fe_super.npy'), dtype=torch.float32)
optim = torch.optim.Adam(nn_model.parameters(), lr=lrate)
data.shape

In [ ]:
# helper functions 

def perturb_input(x, t, noise):
    """
    add noise to specific level
    """
    return ab_t.sqrt()[t, None, None, None, None] * x + (1 - ab_t).sqrt()[t, None, None, None, None] * noise

In [ ]:
# train model

nn_model.train()

for ep in range(n_epoch):
    print(f'epoch {ep}')

    optim.param_groups[0]['lr'] = lrate*(1-ep/n_epoch)

    pbar = tqdm(data)

    for x in pbar:
        optim.zero_grad()
        x = x.to(device)

        noise = torch.rand_like(x)
        t = torch.randint(1, timesteps +1, (x.shape[0],)).to(device)
        x_pert = perturb_input(x, t, noise)

        pred_noise = nn_model(x_pert, t / timesteps)

        loss = F.mse_loss(pred_noise, noise.reshape(pred_noise.shape), reduction = "mean")
        print(loss)
        loss.backward()

        optim.step()

    # save model periodically
    if ep%20==0 or ep == int(n_epoch-1):
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        torch.save(nn_model.state_dict(), save_dir + f"model_{ep}.pth")
        print('saved model at ' + save_dir + f"model_{ep}.pth")

In [ ]:
# helper function; removes the predicted noise (but adds some noise back in to avoid collapse)
def denoise_add_noise(x, t, pred_noise, z=None):
    if z is None:
        z = torch.randn_like(x)
    noise = b_t.sqrt()[t] * z
    mean = (x - pred_noise * ((1 - a_t[t]) / (1 - ab_t[t]).sqrt())) / a_t[t].sqrt()
    return mean + noise

In [ ]:
# load in model weights and set to eval mode
nn_model.load_state_dict(torch.load(f"{save_dir}/model_99.pth", map_location=device))
nn_model.eval()
print("Loaded in Model")

In [ ]:
# sample using standard algorithm
@torch.no_grad()
def sample_ddpm(n_sample, save_rate=50):
    # x_T ~ N(0, 1), sample initial noise
    samples = torch.randn(n_sample, 1, height, height, height).to(device)  

    # array to keep track of generated steps for plotting
    intermediate = [] 
    for i in range(timesteps, 0, -1):
        print(f'sampling timestep {i}', end='\r')

        # reshape time tensor
        t = torch.tensor([i / timesteps])[:, None, None, None, None].to(device)

        # sample some random noise to inject back in. For i = 1, don't add back in noise
        z = torch.randn_like(samples) if i > 1 else 0

        eps = nn_model(samples, t)    # predict noise e_(x_t,t)
        samples = denoise_add_noise(samples, i, eps, z)
        
        if i % save_rate ==0 or i==timesteps or i<8:
            intermediate.append(samples.detach().cpu().numpy())

            x_project = np.mean(samples.cpu().squeeze().detach().numpy(), axis=0)
            plt.imshow(x_project)
            plt.colorbar()
            plt.show()

    intermediate = np.stack(intermediate)
    return samples, intermediate

In [ ]:
# visualize samples
plt.clf()
samples, intermediate_ddpm = sample_ddpm(1)

In [ ]:
x_project = np.mean(intermediate_ddpm[26].squeeze(), axis=0)
plt.imshow(x_project)
plt.colorbar()